# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Exploring and clustering the neighborhoods in Toronto

### Part 1

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
html_doc = response.text
soup = BeautifulSoup(html_doc, 'html.parser')
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        # cell['Neighborhood'] = (((((row.span.text).split('(')[1:]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        borough_list = (row.span.text).split('(')[1:]
        borough_list = [x.replace(' /', ',').replace(')',' ').strip(' ') for x in borough_list]
        cell['Neighborhood'] = ', '.join(borough_list)
        table_contents.append(cell)

In [3]:
df = pd.DataFrame(table_contents)
assert(df[df['Borough'] == 'Not assigned'].shape == (0, 3))


In [4]:
df.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [9]:
#dimensi
df.shape

(103, 3)

### Part 2

In [16]:
!conda install -c conda-forge geocoder -y

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_1
  ratelim            conda-forge/noarch::ratelim-0.1.6-py_2



ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [17]:
import geocoder
import requests
from io import StringIO

In [18]:
def get_coords(place, postal_code):
    
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, {}'.format(postal_code, place))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

In [ ]:
#get_coords('Scarborough', 'M1B')
#geocoder doesn't work

In [20]:
# using csv file
zip_code_url = 'http://cocl.us/Geospatial_data'
s=requests.get(zip_code_url).content
df_geo=pd.read_csv(StringIO(s.decode('utf-8')))

In [21]:
df_geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [22]:
df = df.join(df_geo.set_index('Postal Code'), on='PostalCode')

In [71]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [23]:
df.shape

(103, 5)